In [1]:
import pandas as pd
import numpy as np
import torch
from basicsr.utils.event_utils import *
import math

/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/tqdm-4.64.1-py3.7.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
txt_file = 'datasets/CED/CED_additional_IR_filter/outdoor_shadow_1_infrared/events/000573.txt'

In [3]:
data = pd.read_csv(txt_file, delim_whitespace=True, header=None, names=['t', 'x', 'y', 'p'], dtype={'t': np.float64, 'x': np.int16, 'y': np.int16, 'p': np.int16})

In [4]:
t = torch.from_numpy(data['t'].values)
x = torch.from_numpy(data['x'].values)
y = torch.from_numpy(data['y'].values)
p = torch.from_numpy(data['p'].values)

In [18]:
print(x.dtype)
print(y.dtype)
print(p.dtype)
print(t.shape)

torch.int16
torch.int16
torch.int16
torch.Size([1])


In [6]:
voxel = events_to_voxel_torch(x, y, t, p, 5, sensor_size=(260, 346))

In [7]:
def voxel_normalization(voxel):
    """
        normalize the voxel same as https://arxiv.org/abs/1912.01584 Section 3.1
        Params:
            voxel: torch.Tensor, shape is [num_bins, H, W]
        
        return:
            normalized voxel
    """
    abs_voxel, _ = torch.sort(torch.abs(voxel).view(-1, 1).squeeze(1))
    first_non_zero_idx = torch.nonzero(abs_voxel)[0].item()
    non_zero_voxel = abs_voxel[first_non_zero_idx:]
    norm_idx = math.floor(non_zero_voxel.shape[0] * 0.98)

    ones = torch.ones_like(voxel)

    normed_voxel = torch.where(voxel >= non_zero_voxel[norm_idx], ones, voxel)
    normed_voxel = torch.where(voxel <= -non_zero_voxel[norm_idx], -ones, voxel)
    normed_voxel = torch.where(torch.abs(voxel) < non_zero_voxel[norm_idx], voxel / non_zero_voxel[norm_idx], voxel)

    return normed_voxel

In [8]:
normed_voxel = voxel_normalization(voxel)

In [9]:
np_voxel = normed_voxel.numpy()

In [10]:
np_voxel.shape

(5, 260, 346)

In [19]:
input = 'datasets/CED/CED_additional_IR_filter/outdoor_jumping_1_infrared'
from glob import glob
import os.path as osp
events_paths = sorted(glob(osp.join(input, 'events/*.txt')))

In [20]:
events_paths[0]

'datasets/CED/CED_additional_IR_filter/outdoor_jumping_1_infrared/events/000000.txt'

In [21]:
osp.splitext('datasets/CED/CED_additional_IR_filter/outdoor_jumping_1_infrared/events/000000.txt')

('datasets/CED/CED_additional_IR_filter/outdoor_jumping_1_infrared/events/000000',
 '.txt')

In [22]:
int('000001')

1

In [36]:
import os
root_path_list = []
h5_path_list = []
folder_path = 'datasets/CED'
for root, subdir, _ in os.walk(folder_path):
    if 'images' in subdir:
        root_path_list.append(root)
        out = osp.join('datasets/CED_h5', osp.basename(root) + '.h5')
        h5_path_list.append(out)

In [37]:
h5_path_list[0]

'datasets/CED_h5/simple_rabbits.h5'

In [38]:
root_path_list[0]

'datasets/CED/CED_simple/simple_rabbits'

In [41]:
pathlist = []
for root, h5 in zip(root_path_list, h5_path_list):
    pathlist.append([root, h5])

In [42]:
pathlist[0]

['datasets/CED/CED_simple/simple_rabbits', 'datasets/CED_h5/simple_rabbits.h5']

## hdf5 analysis

In [12]:
import h5py

path = 'datasets/CED_h5/calib_outdoor.h5'
file = h5py.File(path, 'r')

In [13]:
file['images/000000'].attrs['timestamp']

'1555332068.1933038'

In [ ]:
file.keys()
print(file['events'].keys())

In [1]:
pathlist = ['datasets/CED/CED_simple/simple_rabbits', 'datasets/CED_h5/simple_rabbits.h5']

In [2]:
from basicsr.utils.hdf5_util import *

/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/tqdm-4.64.1-py3.7.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
write_h5_worker(pathlist)

Package simple_rabbits.h5 finished.


In [1]:
from basicsr.utils import scandir
import os
from glob import glob
import  os.path as osp

h5_path = glob(osp.join('datasets/CED_h5', '*.h5'))

/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/tqdm-4.64.1-py3.7.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import h5py
file = h5py.File('datasets/CED_h5/driving_city_5.h5', 'r')
len(file['voxel'].keys())
file['voxel/004318'].attrs['is_empty']

False

In [2]:
from basicsr.utils.hdf5_util import write_h5_worker
pathlist = [
    'datasets/CED/HR/CED_driving/driving_city_sun_2',
    'datasets/CED_h5/HR/driving_city_sun_2.h5'
]
write_h5_worker(pathlist)

datasets/CED/HR/CED_driving/driving_city_sun_2/events/001765.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/driving_city_sun_2/events/001766.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/driving_city_sun_2/events/001767.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/driving_city_sun_2/events/001768.txt is empty, will write zero voxel
datasets/CED/HR/CED_driving/driving_city_sun_2/events/001769.txt is empty, will write zero voxel


KeyboardInterrupt: 

In [8]:
from basicsr.utils.hdf5_util import write_h5_worker
pathlist = [
    'datasets/CED/HR/CED_people/people_dynamic_wave',
    'datasets/CED_h5/HR/people_dynamic_wave.h5'
]
write_h5_worker(pathlist)

/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Package people_dynamic_wave.h5 finished.


In [1]:
from basicsr.utils.hdf5_util import write_h5_worker
pathlist = [
    'datasets/CED/HR/CED_simple/simple_jenga_destroy',
    'datasets/CED_h5/HR/simple_jenga_destroy.h5'
]
write_h5_worker(pathlist)


/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/tqdm-4.64.1-py3.7.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Package simple_jenga_destroy.h5 finished.


In [13]:
import pandas as pd
import numpy as np

txt_file = 'datasets/CED/CED_driving/driving_city_5/events/004318.txt'
data = pd.read_csv(txt_file, delim_whitespace=True, header=None, names=['t', 'x', 'y', 'p'], dtype={'t': np.float64, 'x': np.int16, 'y': np.int16, 'p': np.int16})

In [14]:
data.empty

False

In [15]:
a = np.zeros((5, 260, 346))

In [16]:
a.shape

(5, 260, 346)

In [1]:
import h5py
root_path = 'datasets/CED_h5/HR'
from glob import glob
import os
import os.path as osp 
h5_list = glob(osp.join(root_path, "*.h5"))

In [2]:
root_path_list = []
h5_path_list = []
folder_path = 'datasets/CED/HR'
for root, subdir, _ in os.walk(folder_path):
    if 'images' in subdir:
        root_path_list.append(root)
        out = osp.join('datasets/CED_h5/HR', osp.basename(root) + '.h5')
        h5_path_list.append(out)

In [3]:
total_path = []
for root, h5 in zip(root_path_list, h5_path_list):
    total_path.append([root, h5])
print(total_path[0])

['datasets/CED/HR/CED_simple/simple_rabbits', 'datasets/CED_h5/HR/simple_rabbits.h5']


In [4]:
cnt = 0
print(len(total_path))
for f in total_path:
    len_ori = len(glob(osp.join(f[0], 'events', "*.txt")))

    file = h5py.File(f[1], 'r')
    event_num1 = len(file['voxels'].keys())
    # if 'voxels' in file.keys():
    #     event_num2 = len(file['voxels'].keys())
    # else:
    #     event_num2 = 0
    event_num = event_num1
    if len_ori != event_num:
        print(f[1], len_ori, event_num)

84


In [15]:
file = h5py.File('datasets/CED_h5/simple_fruit_fast.h5', 'r')
print(len(file['voxel'].keys()))

932


## Cp HR/timestamp.txt to LR/timestamp.txt

In [1]:
import os
import os.path as osp

In [2]:
HR_path = 'datasets/CED/HR/'
LR_path = 'datasets/CED/LRx4'
for root, subdir, files in os.walk(HR_path):
    if 'timestamp.txt' in files:
        folder = root.split(HR_path)[1]
        src = osp.join(HR_path, folder, 'timestamp.txt')
        dst = osp.join(LR_path, folder)
        os.system(f"cp -rf {src} {dst}")

## get hr voxel and use bicubic to downsample

In [1]:
import os
import os.path as osp
import numpy as np

folder_path = 'datasets/CED/LR'
root_path_list = []
h5_hr_path_list = []
h5_lr_path_list = []
for root, subdir, _ in os.walk(folder_path):
    if 'images' in subdir:
        root_path_list.append(root)
        h5_hr_path_list.append(osp.join('datasets/CED_h5/HR', osp.basename(root) + '.h5'))
        h5_lr_path_list.append(osp.join('datasets/CED_h5/LR', osp.basename(root) + '.h5'))

In [2]:
h5_hr_path_list[0]

'datasets/CED_h5/HR/simple_rabbits.h5'

In [3]:
import h5py

file = h5py.File(h5_hr_path_list[0], 'r')


In [16]:
file.attrs.keys()

<KeysViewHDF5 ['duration', 'num_events', 'num_imgs', 'sensor_resolution', 't0', 'tk']>

In [18]:
file['voxels/000000'].attrs['is_empty']

False

In [5]:
voxel_ex.shape

(5, 260, 346)

In [8]:
import sys
sys.path.append('./scripts/data_preparation')
from scripts.data_preparation.pyResize import DownSample, UpSample, imresize

/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/tqdm-4.64.1-py3.7.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
voxel_lr = imresize(voxel_ex, 0.5)

In [11]:
voxel_lr.shape

(3, 130, 346)

In [9]:
import torch
voxel_tensor = torch.from_numpy(voxel_ex).unsqueeze(0)

In [14]:
lr = torch.nn.functional.interpolate(input = voxel_tensor, scale_factor = 0.5, mode = 'bicubic').squeeze(0).numpy()

/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/torch/nn/functional.py:3680: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "


In [15]:
lr.shape

(5, 130, 173)

## test lr h5 file

In [10]:
import h5py
from basicsr.utils.hdf5_util import *

pathlist = [
    'datasets/CED/LR/CED_simple/simple_carpet',
    'datasets/CED_h5/HR/simple_carpet.h5',
    'datasets/CED_h5/LR/simple_carpet.h5'
]

write_lr_h5_worker(pathlist)


/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/home/kaidc/miniconda3/envs/e2vsr/lib/python3.7/site-packages/torch/nn/functional.py:3680: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "


Package simple_carpet.h5 finished.


In [11]:
import h5py
file = h5py.File('datasets/CED_h5/HR/driving_city_4.h5', 'r')
file.attrs.keys()

<KeysViewHDF5 ['duration', 'num_events', 'num_imgs', 'sensor_resolution', 't0', 'tk']>

In [15]:
file['voxels/000000'].shape

(5, 260, 346)

In [13]:
file.attrs['num_events']

17514

In [12]:
import os

txt_path = '/home/kaidc/code/e2vsr/basicsr/data/meta_info/CED_tmp.txt'

with open(txt_path, 'r') as f:
    str_temp = f.read().splitlines()
    print(str_temp)

['people/people_dynamic_wave/images 759 (260,346,3)', 'indoors/indoors_foosball_2/images 269 (260,346,3)', 'simple/simple_wires_2/images 552 (260,346,3)', 'people/people_dynamic_dancing/images 1175 (260,346,3)', 'people/people_dynamic_jumping/images 792 (260,346,3)', 'simple/simple_fruit_fast/images 933 (260,346,3)', 'additional_IR_filter/outdoor_jumping_infrared_2/images 665 (260,346,3)', 'simple/simple_carpet_fast/images 602 (260,346,3)', 'people/people_dynamic_armroll/images 792 (260,346,3)', 'indoors/indoors_kitchen_2/images 635 (260,346,3)', 'people/people_dynamic_sitting/images 1075 (260,346,3)']


In [13]:
to_write = str_temp[0].split(' ')[0].split('/')[1] + ' ' + str_temp[0].split(' ')[1]
print(to_write)

people_dynamic_wave 759


In [14]:
txt_path = '/home/kaidc/code/e2vsr/basicsr/data/meta_info/CED_h5_test.txt'

with open(txt_path, 'w') as f:
    for i in range(len(str_temp)):
        to_write = str_temp[i].split(' ')[0].split('/')[1] + '.h5 ' + str_temp[i].split(' ')[1] + '\n'
        f.write(to_write)

In [25]:
txt_path = '/home/kaidc/code/e2vsr/basicsr/data/meta_info/CED_h5_train_2.txt'

with open(txt_path, 'w') as f:
    for i in range(len(str_temp)):
        f.write(str_temp[i].split(' ')[0].split('/')[1] + '.h5\n')

In [7]:
import os
import os.path as osp
file = 'basicsr/data/meta_info/CED_h5_train.txt'

keys = []
frame_num = {}
with open(file, 'r') as fin:
    for line in fin:
        name, num = line.split(' ')
        keys.extend([f'{name}/{i:06d}' for i in range(int(num))])
        print(osp.dirname("simple_flowers_infrared.h5"))
        frame_num[osp.dirname(name)] = num.strip()

In [8]:
frame_num.get('simple_flowers_infrared')

In [5]:
frame_num = {}
name = 'simple'
num = '10'
frame_num[name] = num
type(frame_num)

dict

In [1]:
import h5py
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [13]:
file = h5py.File('datasets/CED_h5/HR/people_dynamic_wave.h5', 'r')

In [14]:
file.keys()

<KeysViewHDF5 ['images', 'voxels']>

In [15]:
img = file['images/000003'][:]

In [19]:
import numpy as np
np_path = 'datasets/CED/HR/CED_people/people_dynamic_wave/images/000000.png'
img_np = cv2.imread(np_path)

In [17]:
(img == img_np).all()

False

In [19]:
cv2.imwrite('tmp/img.png', img)

True

In [10]:
import lmdb
path = '/home/kaidc/data/VSR/CED_lmdb/train_sharp_with_val.lmdb'

env = lmdb.open(path)

In [11]:
txn = env.begin()

In [12]:
img2 = txn.get(str('people/people_dynamic_wave/images/000000').encode())

In [13]:
type(img2)

bytes

In [14]:
from basicsr.utils.img_util import imfrombytes
img2 = imfrombytes(img2, float32=True)

In [15]:
import numpy as np
np.max(img2)

0.99607843

In [13]:
cv2.imwrite('tmp/img2.png', img2)

True

In [2]:
import h5py

file = h5py.File('datasets/CED_h5/LRx4/calib_fluorescent.h5', 'r')

In [3]:
file.keys()

<KeysViewHDF5 ['images', 'voxels']>

In [4]:
file.attrs.keys()

<KeysViewHDF5 ['duration', 'num_events', 'num_imgs', 'sensor_resolution', 't0', 'tk']>

In [5]:
file['voxels/000000']

(5, 65, 86)